<a href="https://colab.research.google.com/github/emanfj/Healthcare-Disparities-Analysis/blob/main/joint_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

have not tested ram crashed ;(

In [1]:
from google.colab import drive,files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# necessary imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
# loading both datasets
healthcare_data = pd.read_csv('/content/drive/MyDrive/processed_census2017.csv')
census_data = pd.read_csv('/content/drive/MyDrive/heart_2022_no_nans.csv')

Identify populations at higher risk for specific health conditions

In [ ]:
# merging datasets on State column
merged_data = healthcare_data.merge(census_data, on='State')

def train_and_evaluate_models(target_variable, features):
  """
  Trains and evaluates different models for a given target variable and feature set.

  Args:
      target_variable: The name of the target variable in the merged data.
      features: A list of feature names to use for training.
  """
  # splitting data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(merged_data[features], merged_data[target_variable], test_size=0.2, random_state=42)

  # defining models and their hyperparameter grids for tuning (adjust as needed)
  models = {
      'Logistic Regression': LogisticRegression(),
      'Decision Tree': DecisionTreeClassifier(max_depth=[3, 5, 7]),
      'Random Forest': RandomForestClassifier(n_estimators=[100, 200], max_depth=[5, 8])
  }

  # performing hyperparameter tuning with GridSearchCV
  for model_name, model in models.items():
    tuned_model = GridSearchCV(model, param_grid=models[model_name], cv=5, scoring='accuracy')
    tuned_model.fit(X_train, y_train)
    best_model = tuned_model.best_estimator_
    accuracy = best_model.score(X_test, y_test)
    print(f"{model_name} Accuracy (Tuned): {accuracy:.2f}")

  # training and evaluate base models (without tuning)
  for model_name, model in models.items():
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"{model_name} Accuracy: {accuracy:.2f}")

# defining features (change after testing)
features = ['Age', 'Race', 'Income', 'Commute_Method']
# defining potential target variables
target_variables = ['Heart_Disease', 'Diabetes', 'Obesity']

# loop through target variables and train models for each
for target_variable in target_variables:
  print(f"\nTarget Variable: {target_variable}")
  train_and_evaluate_models(target_variable, features)
